In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv(r'C:\Users\ASUS\OneDrive\Desktop\code\serious_projects\model_monitering\data\Loan_approval_data_2025.csv')
print(data.shape)
df = data.copy()
df[:5]

(50000, 20)


,customer_id,age,occupation_status,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,product_type,loan_intent,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
0,CUST100000,40,Employed,17.2,25579,692,5.3,895,10820,0,0,0,Credit Card,Business,600,17.02,0.423,0.023,0.008,1
1,CUST100001,33,Employed,7.3,43087,627,3.5,169,16550,0,1,0,Personal Loan,Home Improvement,53300,14.10,0.384,1.237,0.412,0
2,CUST100002,42,Student,1.1,20840,689,8.4,17,7852,0,0,0,Credit Card,Debt Consolidation,2100,18.33,0.377,0.101,0.034,1
3,CUST100003,53,Student,0.5,29147,692,9.8,1480,11603,0,1,0,Credit Card,Business,2900,18.74,0.398,0.099,0.033,1
4,CUST100004,32,Employed,12.5,63657,630,7.2,209,12424,0,0,0,Personal Loan,Education,99600,13.92,0.195,1.565,0.522,1


In [3]:
temp = pd.get_dummies(df['occupation_status'],dtype=float)
df=pd.concat([df,temp],axis=1)
df.drop(['occupation_status'],axis=1,inplace=True)
temp = pd.get_dummies(df['product_type'],dtype=float)
df=pd.concat([df,temp],axis=1)
df.drop(['product_type'],axis=1,inplace=True)
temp = pd.get_dummies(df['loan_intent'],dtype=float)
df=pd.concat([df,temp],axis=1)
df.drop(['loan_intent'],axis=1,inplace=True)
df.drop(['customer_id'],axis=1,inplace=True)
df[:5]


,age,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,...,Student,Credit Card,Line of Credit,Personal Loan,Business,Debt Consolidation,Education,Home Improvement,Medical,Personal
0,40,17.2,25579,692,5.3,895,10820,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,33,7.3,43087,627,3.5,169,16550,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,42,1.1,20840,689,8.4,17,7852,0,0,0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,53,0.5,29147,692,9.8,1480,11603,0,1,0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,32,12.5,63657,630,7.2,209,12424,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [4]:
data_numeric = df.copy()
y=data_numeric['loan_status']
data_numeric.drop(['loan_status'],axis=1,inplace=True)
x=data_numeric.copy()

In [5]:
sc = StandardScaler()
x_trian,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)
x_trian = sc.fit_transform(x_trian)
X_test = sc.transform(X_test) 

In [6]:
from sklearn.svm import SVC
model = SVC(verbose=True).fit(x_trian,y_train)
y_pred = model.predict(X_test)

[LibSVM]

In [7]:
precision_score(y_test,y_pred)

0.9105603448275862

In [8]:
confusion_matrix(y_test,y_pred)

array([[6017,  747],
       [ 631, 7605]], dtype=int64)

In [9]:
from minio import Minio
import io
import pickle

client = Minio("minio:9000", access_key="minio", secret_key="minio123", secure=False)
pickle_bytes = pickle.dumps(model)
client.put_object("ml-models", "baseline.pkl", io.BytesIO(pickle_bytes), len(pickle_bytes))


MaxRetryError: HTTPConnectionPool(host='minio', port=9000): Max retries exceeded with url: /ml-models?location= (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x0000015B5FF7A2D0>: Failed to resolve 'minio' ([Errno 11001] getaddrinfo failed)"))